# Handwritten Character Recognition - Enhanced Gradio Application

This notebook launches an interactive Gradio application for handwritten character recognition. It allows users to select a model, and then either draw characters, upload an image, or use a webcam for character recognition. The application displays the recognized text and visualizes intermediate processing steps.

**Prerequisites:**
1.  **Trained Models:** Ensure you have trained models saved in the appropriate checkpoint directories (e.g., `./model_checkpoints/cnn/best_model.pth` or `./model_checkpoints/vgg/best_model.pth`). The `training.ipynb` notebook should produce these.
2.  **Dataset for Class Labels:** The path to the root of the training dataset is needed to derive the class labels. Update `DATA_ROOT_FOR_LABELS` if your dataset is located elsewhere.

## 1. Imports and Setup

In [ ]:
# General utilities
import os
import random
import copy
import traceback
import subprocess # For robust pip install

# Image processing and display
import matplotlib
# Ensure a non-interactive backend is used if running in a headless environment for plt.figure creation for Gradio gallery
if os.environ.get('DISPLAY','') == '' and os.name != 'posix':
    print("Setting Matplotlib backend to 'Agg' for non-interactive environment.")
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from PIL import Image, ImageOps 
import cv2 
import numpy as np

# PyTorch essentials
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torchvision.models as models
from pathlib import Path 
import matplotlib.patches as patches 

# Gradio for the web application
try:
    import gradio as gr
except ImportError:
    print("Gradio not installed. Installing gradio==3.50.2...") # Pinned version for stability
    subprocess.check_call(["pip", "install", "gradio==3.50.2", "-q"])
    import gradio as gr

print(f"Gradio version: {gr.__version__}")
print(f"PyTorch version: {torch.__version__}")
import torchvision
print(f"Torchvision version: {torchvision.__version__}")

## 2. Device Configuration

In [ ]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

## 3. Model Architectures
These must match the definitions used during training to load the weights correctly.

In [ ]:
class LetterCNN64(nn.Module):
    def __init__(self, num_classes, device='cpu'): # Added device parameter
        super(LetterCNN64, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)
        self.to(device) # Move model to device

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return x

class ImprovedLetterCNN(nn.Module):
    def __init__(self, num_classes, device='cpu'): # Added device parameter
        super(ImprovedLetterCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.bn_fc1 = nn.BatchNorm1d(1024)
        self.relu_fc1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.bn_fc2 = nn.BatchNorm1d(512)
        self.relu_fc2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, num_classes)
        self.to(device) # Move model to device

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = self.pool4(self.relu4(self.bn4(self.conv4(x))))
        x = x.view(-1, 256 * 4 * 4)
        x = self.dropout1(self.relu_fc1(self.bn_fc1(self.fc1(x))))
        x = self.dropout2(self.relu_fc2(self.bn_fc2(self.fc2(x))))
        x = self.fc3(x)
        return x

class VGG19HandwritingModel(nn.Module):
    def __init__(self, num_classes, device, pretrained=True):
        super(VGG19HandwritingModel, self).__init__()
        self.device = device 
        vgg19 = models.vgg19_bn(weights=models.VGG19_BN_Weights.IMAGENET1K_V1 if pretrained else None)
        self.features = vgg19.features
        if pretrained:
            for param in self.features.parameters():
                param.requires_grad = False
        num_features_output = 512 * 2 * 2 
        self.classifier = nn.Sequential(
            nn.Linear(num_features_output, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(2048, num_classes)
        )
        self._initialize_weights()
        self.to(self.device) # Ensure the entire model is on the correct device

    def _initialize_weights(self):
        for m in self.classifier.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

## 4. Application Utilities

In [ ]:
def load_model_for_app(model_class_fn, num_classes, checkpoint_path, model_name_display, current_device):
    """Loads a specified model class for the Gradio app, ensuring it's on the correct device."""
    model_instance = model_class_fn(num_classes=num_classes, device=current_device)
    # Model is moved to device within its __init__ method.
    
    if not os.path.exists(checkpoint_path):
        print(f"WARNING: Checkpoint for {model_name_display} not found at {checkpoint_path}. Model will be uninitialized.")
        return model_instance 
    try:
        checkpoint = torch.load(checkpoint_path, map_location=current_device)
        state_dict = checkpoint.get('model_state_dict', checkpoint) # Handle older checkpoints that saved the state_dict directly
        model_instance.load_state_dict(state_dict)
        model_instance.eval()
        print(f"{model_name_display} model loaded successfully from {checkpoint_path} and set to {current_device}.")
    except Exception as e:
        print(f"ERROR loading {model_name_display} from {checkpoint_path}: {e}. Model remains uninitialized or partially loaded.")
        traceback.print_exc()
    return model_instance

def get_class_labels_from_dir(data_root_dir):
    if not os.path.exists(data_root_dir) or not os.path.isdir(data_root_dir):
        print(f"Warning: Data root for labels '{data_root_dir}' not found or not a directory.")
        return []
    try:
        temp_dataset = datasets.ImageFolder(root=data_root_dir)
        if not temp_dataset.classes: print(f"Warning: No classes found in '{data_root_dir}'. Check dataset structure.")
        return temp_dataset.classes
    except Exception as e: print(f"Error getting class labels from '{data_root_dir}': {e}"); return []

def prepare_char_roi_for_model(char_pil_image, image_size=(64,64), spacing=12):
    """Prepares a single character ROI (PIL Image, L mode) for 3-channel model input."""
    target_s = image_size[0] - spacing 
    w, h = char_pil_image.size
    if w == 0 or h == 0: return None
    new_w, new_h = (target_s, int((h/w) * target_s)) if w > h else (int((w/h) * target_s), target_s)
    new_w, new_h = max(1, new_w), max(1, new_h)
    resized = char_pil_image.resize((new_w, new_h), Image.LANCZOS)
    padded = Image.new('L', (target_s, target_s), 255) # White background
    px, py = (target_s - new_w) // 2, (target_s - new_h) // 2
    padded.paste(resized, (px,py))
    final_img = Image.new('L', image_size, 255); spacing_offset = spacing//2
    final_img.paste(padded, (spacing_offset, spacing_offset))
    transform_to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3,1,1) if x.size(0)==1 else x),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])
    return transform_to_tensor(final_img).unsqueeze(0)

## 5. Gradio Application Setup and Core Logic

In [ ]:
# --- Load Models and Class Labels --- #
MODEL_CKPT_DIR_CNN = './model_checkpoints/cnn/'
MODEL_CKPT_DIR_VGG = './model_checkpoints/vgg/'
DATA_ROOT_FOR_LABELS = "./datasets/handwritten-english/augmented_images/augmented_images1" # USER: Update if needed

print(f"Attempting to load class labels from: {DATA_ROOT_FOR_LABELS}")
app_class_labels = get_class_labels_from_dir(DATA_ROOT_FOR_LABELS)
if not app_class_labels:
    print("CRITICAL WARNING: Class labels could not be loaded. Using a fallback list of 62 common chars.")
    app_class_labels = ([str(i) for i in range(10)] + 
                        [chr(ord('A')+i) for i in range(26)] + 
                        [chr(ord('a')+i) for i in range(26)])
app_num_classes = len(app_class_labels)
print(f"Loaded {app_num_classes} class labels. First 5: {app_class_labels[:5]}")

loaded_models_dict = {}
available_model_names_list = []

if app_num_classes > 0:
    cnn_checkpoint_file = os.path.join(MODEL_CKPT_DIR_CNN, 'best_model.pth')
    if os.path.exists(cnn_checkpoint_file):
        cnn_model_instance = load_model_for_app(ImprovedLetterCNN, app_num_classes, cnn_checkpoint_file, "ImprovedCNN", device)
        if cnn_model_instance: loaded_models_dict["ImprovedCNN"] = cnn_model_instance; available_model_names_list.append("ImprovedCNN")
    else:
        print(f"Checkpoint for ImprovedCNN not found at {cnn_checkpoint_file}")

    vgg_checkpoint_file = os.path.join(MODEL_CKPT_DIR_VGG, 'best_model.pth')
    if os.path.exists(vgg_checkpoint_file):
        vgg_model_instance = load_model_for_app(VGG19HandwritingModel, app_num_classes, vgg_checkpoint_file, "VGG19", device)
        if vgg_model_instance: loaded_models_dict["VGG19"] = vgg_model_instance; available_model_names_list.append("VGG19")
    else:
        print(f"Checkpoint for VGG19 not found at {vgg_checkpoint_file}")
else:
    print("CRITICAL: Number of classes is 0, cannot load models.")

if not loaded_models_dict and app_num_classes > 0:
    print("WARNING: No trained models found. Creating dummy models to allow UI to load. Predictions will be random.")
    dummy_cnn = ImprovedLetterCNN(num_classes=app_num_classes, device=device)
    loaded_models_dict["ImprovedCNN (Dummy)"] = dummy_cnn
    if not available_model_names_list: available_model_names_list.append("ImprovedCNN (Dummy)")

default_model_choice_app = available_model_names_list[0] if available_model_names_list else "No models available"
print(f"Available models for Gradio: {available_model_names_list}")
print(f"Default model for Gradio: {default_model_choice_app}")

In [ ]:
def create_image_grid_for_gradio(images_data_with_labels, title):
    """Helper to create a grid of images for Gradio gallery. Expects list of (image_array, label) tuples."""
    if not images_data_with_labels: return None
    cols = max(1, min(len(images_data_with_labels), 4))
    rows = (len(images_data_with_labels) + cols - 1) // cols
    fig_w, fig_h = cols * 2.5, rows * 2.7 
    if os.environ.get('DISPLAY','') == '' and os.name != 'posix': 
        try: matplotlib.use('Agg')
        except Exception as e: print(f"Matplotlib backend switch failed: {e}")
        
    fig = plt.figure(figsize=(fig_w, fig_h)); 
    if title: fig.suptitle(title, fontsize=10)
    for i, (img_to_show, img_label) in enumerate(images_data_with_labels):
        ax = fig.add_subplot(rows, cols, i + 1)
        ax.set_title(img_label, fontsize=8)
        cmap_val = 'gray' if len(img_to_show.shape) == 2 or (len(img_to_show.shape) == 3 and img_to_show.shape[2] == 1) else None
        ax.imshow(img_to_show.squeeze(), cmap=cmap_val); ax.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95 if title else 1])
    
    fig.canvas.draw()
    grid_img_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    grid_img_np = grid_img_np.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    return grid_img_np

def process_image_for_gradio_tabs(input_pil_image, selected_model_name_str, is_single_char_mode=False, spacing=12):
    gallery_output_images = [] # This will store the actual NumPy arrays for the gallery
    
    if input_pil_image is None: 
        return "No image provided.", [np.zeros((100,100,3), dtype=np.uint8)]

    current_model_to_use = loaded_models_dict.get(selected_model_name_str)
    if current_model_to_use is None or selected_model_name_str == "No models available":
        return f"Error: Model '{selected_model_name_str}' not loaded/available.", [np.array(input_pil_image.convert('RGB'))]

    try:
        input_image_np_rgb = np.array(input_pil_image.convert('RGB'))
        gallery_output_images.append(input_image_np_rgb.copy()) # Original
        
        gray_image = cv2.cvtColor(input_image_np_rgb, cv2.COLOR_RGB2GRAY)
        gallery_output_images.append(cv2.cvtColor(gray_image.copy(), cv2.COLOR_GRAY2RGB)) # Grayscale

        if is_single_char_mode:
            # Sketchpad is black ink on white bg. Model expects black char on white bg for ROI processing.
            _, char_roi_binarized_inv = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            char_roi_binarized = 255 - char_roi_binarized_inv # Black char on white
            gallery_output_images.append(cv2.cvtColor(char_roi_binarized.copy(),cv2.COLOR_GRAY2RGB)) # Binarized ROI
            
            char_pil = Image.fromarray(char_roi_binarized).convert('L')
            tensor_input = prepare_char_roi_for_model(char_pil, spacing=0)
            if tensor_input is None: return "Error: Could not process drawn character.", gallery_output_images
            
            model_input_vis_np = tensor_input.squeeze(0).cpu().permute(1,2,0).numpy()
            mean, std = np.array([0.485,0.456,0.406]), np.array([0.229,0.224,0.225])
            model_input_vis_np=(model_input_vis_np*std)+mean; model_input_vis_np=np.clip(model_input_vis_np,0,1)
            gallery_output_images.append((model_input_vis_np * 255).astype(np.uint8)) # Processed for Model

            with torch.no_grad():
                outputs = current_model_to_use(tensor_input.to(device))
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                confidence, predicted_idx = torch.max(probabilities, 1)
                char_name = app_class_labels[predicted_idx.item()]
            return f"{char_name} (Conf: {confidence.item()*100:.1f}%)", gallery_output_images

        # Multi-character segmentation logic
        _, binary_for_contours = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        gallery_output_images.append(cv2.cvtColor(binary_for_contours.copy(), cv2.COLOR_GRAY2RGB)) # Binary for Contours
        
        _, binary_for_model_rois = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) # Black text on white for ROIs
        gallery_output_images.append(cv2.cvtColor(binary_for_model_rois.copy(), cv2.COLOR_GRAY2RGB)) # Binary for ROIs

        contours, _ = cv2.findContours(binary_for_contours, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contour_img_viz = input_image_np_rgb.copy(); cv2.drawContours(contour_img_viz, contours, -1, (0,255,0), 2)
        gallery_output_images.append(contour_img_viz) # Contours on Original
        if not contours: return "No contours found.", gallery_output_images

        letter_contours = sorted([c for c in contours if cv2.contourArea(c)>20], key=lambda ctr:cv2.boundingRect(ctr)[0])
        if not letter_contours: return "No significant contours found.", gallery_output_images
        
        rois_for_grid_visualization = [] # List of (numpy_array, label_str) for create_image_grid_for_gradio
        recognized_chars_list = []
        output_image_with_boxes = input_image_np_rgb.copy()

        for i, contour in enumerate(letter_contours):
            x, y, w, h = cv2.boundingRect(contour)
            if w < 5 or h < 5: continue
            char_roi_binarized_np = binary_for_model_rois[y:y+h, x:x+w]
            char_pil = Image.fromarray(char_roi_binarized_np).convert('L')
            tensor_input = prepare_char_roi_for_model(char_pil, spacing=spacing)
            if tensor_input is None: continue
            
            model_input_vis_np = tensor_input.squeeze(0).cpu().permute(1,2,0).numpy()
            mean, std = np.array([0.485,0.456,0.406]), np.array([0.229,0.224,0.225])
            model_input_vis_np=(model_input_vis_np*std)+mean; model_input_vis_np=np.clip(model_input_vis_np,0,1)
            rois_for_grid_visualization.append( ( (model_input_vis_np*255).astype(np.uint8), f"ROI {i}" ) )

            with torch.no_grad():
                outputs = current_model_to_use(tensor_input.to(device))
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                confidence, predicted_idx = torch.max(probabilities, 1)
                char_name = app_class_labels[predicted_idx.item()]
            recognized_chars_list.append(char_name)
            cv2.rectangle(output_image_with_boxes, (x,y), (x+w,y+h), (0,0,255), 2)
            cv2.putText(output_image_with_boxes, f"{char_name} ({confidence.item()*100:.0f}%)", (x,y-10 if y-10 > 10 else y+h+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0),1)
        
        if rois_for_grid_visualization:
            rois_grid_img = create_image_grid_for_gradio(rois_for_grid_visualization, "Processed Segments for Model")
            if rois_grid_img is not None: gallery_output_images.append(rois_grid_img)
        
        gallery_output_images.append(output_image_with_boxes) # Final Predictions
        return "".join(recognized_chars_list), gallery_output_images

    except Exception as e:
        print(f"Error in Gradio processing: {e}")
        traceback.print_exc()
        error_img_placeholder = np.zeros((100,300,3),dtype=np.uint8); cv2.putText(error_img_placeholder,"Processing Error",(10,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),1)
        return f"Error: {str(e)}", [error_img_placeholder]

## 6. Launch Gradio Application

In [ ]:
def launch_enhanced_gradio_app():
    if not app_class_labels or app_num_classes == 0:
        gr.Warning("CRITICAL: Class labels could not be loaded or are empty. App functionality will be severely limited.")
    if not loaded_models_dict or default_model_choice_app == "No models available":
        gr.Warning("CRITICAL: No models could be loaded. Predictions will not work. Ensure models are trained and paths are correct.")

    with gr.Blocks(title="Handwritten Character Recognition App", theme=gr.themes.Soft(primary_hue=gr.themes.colors.blue, secondary_hue=gr.themes.colors.indigo)) as app_interface:
        gr.Markdown("<h1><center>Handwritten Character Recognition App</center></h1>")
        gr.Markdown("Select a model, then provide an image via drawing, uploading, or webcam.")

        with gr.Row():
            model_selector_dropdown = gr.Dropdown(choices=available_model_names_list, value=default_model_choice_app, label="Select Recognition Model")
        
        output_text_display = gr.Textbox(label="Recognized Text (Multi-Character)", lines=1, placeholder="Recognized characters will appear here...")
        # For single character, prediction will be shown in a Label within its tab.
        gallery_display = gr.Gallery(label="Processing Visualizations", columns=3, height=600, object_fit="contain", preview=True, show_label=False)

        with gr.Tabs():
            with gr.TabItem("✏️ Draw Single Character"):
                with gr.Row():
                    sketch_input_single = gr.Sketchpad(type="pil", label="Draw Single Character Here", image_mode="RGB", shape=(280,280), 
                                                         invert_colors=False, brush_radius=10, brush_color="#000000", background_color="#FFFFFF", 
                                                         show_label=True, elem_id="sketchpad_single")
                    with gr.Column(scale=1):
                        single_char_pred_output = gr.Label(label="Prediction") 
                        single_char_processed_img_display = gr.Image(label="Processed for Model", type="numpy", width=128, height=128, image_mode="RGB")
                single_char_button = gr.Button("Recognize Single Drawn Character", variant="primary")
                
                def handle_single_char_draw_tab(sketch_pil, model_name):
                    if sketch_pil is None: return "Draw a character.", None, []
                    text_out, gallery_numpy_imgs_list = process_image_for_gradio_tabs(sketch_pil, model_name, is_single_char_mode=True)
                    # For the small display next to sketchpad, show the last image (processed for model)
                    processed_img_for_small_display = gallery_numpy_imgs_list[-1] if gallery_numpy_imgs_list and gallery_numpy_imgs_list[-1] is not None else None
                    return text_out, processed_img_for_small_display, gallery_numpy_imgs_list

                single_char_button.click(fn=handle_single_char_draw_tab, 
                                         inputs=[sketch_input_single, model_selector_dropdown], 
                                         outputs=[single_char_pred_output, single_char_processed_img_display, gallery_display])

            with gr.TabItem("🖼️ Upload Multi-Character Image"):
                upload_input_multi = gr.Image(type="pil", label="Upload Image (Multiple Characters)", image_mode="RGB")
                upload_button_multi = gr.Button("Recognize Uploaded Image", variant="primary")
                upload_button_multi.click(lambda img, model_name: process_image_for_gradio_tabs(img, model_name, is_single_char_mode=False), 
                                          inputs=[upload_input_multi, model_selector_dropdown], 
                                          outputs=[output_text_display, gallery_display])

            with gr.TabItem("📷 Webcam Capture"):
                webcam_input_multi = gr.Image(source="webcam", type="pil", label="Capture from Webcam", image_mode="RGB")
                webcam_button_multi = gr.Button("Recognize from Webcam", variant="primary")
                webcam_button_multi.click(lambda img, model_name: process_image_for_gradio_tabs(img, model_name, is_single_char_mode=False), 
                                         inputs=[webcam_input_multi, model_selector_dropdown], 
                                         outputs=[output_text_display, gallery_display])

        gr.Markdown("--- ")
        gr.Markdown("**Notes:**\n"+
                    "- **Model Selection:** Choose a model before processing an image. Models are loaded from `./model_checkpoints/`\n"+
                    "- **Drawing Single Char:** Draw clearly in the center. The sketchpad is set to black ink on white background.\n"+
                    "- **Uploading/Webcam (Multi-Char):** Clear, well-lit images with good contrast work best. Ensure adequate spacing between characters if possible.\n"+
                    "- **Segmentation:** This is a complex step. Results depend on handwriting clarity and image quality.")
    
    app_interface.launch(share=True, debug=True) # Debug=True for more detailed error messages during development

# Automatically launch if in a Jupyter-like environment and components are ready.
if __name__ == '__main__' and ('ipykernel' in __import__('sys').modules):
    if not app_class_labels or app_num_classes == 0:
        print("CRITICAL WARNING: Class labels could not be loaded/are empty. Gradio app functionality will be severely limited.")
    elif not loaded_models_dict or default_model_choice_app == "No models available":
        print("CRITICAL WARNING: No models could be loaded. Predictions will not work. Gradio app may be non-functional. Ensure models are trained and paths are correct.")
    else:
        print("Attempting to launch Gradio app...")
    launch_enhanced_gradio_app()